# Test `scan` and `apply_layers` in Llama 3

Hugging Face usage follows https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

To test scan, we need to use a custom modification of the transformer repo:
https://github.com/tengyifei/transformers/tree/test-scan

In [1]:
%env PJRT_DEVICE=TPU
%env XLA_USE_SPMD=1

env: PJRT_DEVICE=TPU
env: XLA_USE_SPMD=1


In [2]:
import torch
import torch_xla
import numpy as np
import transformers

/workspaces/torch/pytorch/xla/torch_xla/__init__.py:250: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def reset_seed():
  torch.random.manual_seed(42)
  torch_xla.manual_seed(42)
  np.random.seed(42)

In [4]:
from datasets import load_dataset

dataset = load_dataset("Salesforce/wikitext", "wikitext-103-v1")

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.bos_token_id = 128000
tokenizer.eos_token_id = 128001
tokenizer.pad_token_id = tokenizer.eos_token_id 

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"], batch_size=1000)

In [6]:
tokenized_datasets.keys()  # type:ignore

dict_keys(['test', 'train', 'validation'])

In [7]:
tokenized_datasets["train"][1].keys()  # type:ignore

dict_keys(['input_ids', 'attention_mask'])

In [8]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
)

In [9]:
lm_datasets["train"][1].keys(), lm_datasets["validation"][1].keys()  # type:ignore

(dict_keys(['input_ids', 'attention_mask', 'labels']),
 dict_keys(['input_ids', 'attention_mask', 'labels']))

In [10]:
len(lm_datasets["validation"])  # type:ignore

3760

In [11]:
from transformers import LlamaConfig, LlamaForCausalLM

reset_seed()

# Define model configuration
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,  # Model size
    num_hidden_layers=32,  # Number of transformer layers
    num_attention_heads=8,  # Number of attention heads
    intermediate_size=1024,  # Size of the hidden feedforward layer
    max_position_embeddings=128,  # Max tokens in a sequence
    use_cache=False,
    unroll_decoders=True,
)

# Instantiate the model
model = LlamaForCausalLM(config)

2024-11-17 07:21:53.603012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731828113.611602 2174382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731828113.614233 2174382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    max_steps=2500,
    save_strategy="no",
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    gradient_accumulation_steps=1,
    fp16=False,
    bf16=False,
    tpu_num_cores=4,
    push_to_hub=False,
)

2024-11-17 08:01:49.210165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731830509.228689 2204723 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731830509.234155 2204723 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"].shuffle(seed=42),  # type:ignore
    eval_dataset=lm_datasets["validation"].shuffle(seed=42),  # type:ignore
    processing_class=tokenizer,
)

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer.train()

NOTE: Using for loop to run decoder layers


Epoch,Training Loss,Validation Loss
0,6.202200,6.074329


TrainOutput(global_step=2500, training_loss=6.7848673828125, metrics={'train_runtime': 828.9795, 'train_samples_per_second': 193.008, 'train_steps_per_second': 3.016, 'total_flos': 1.83650746368e+16, 'train_loss': 6.7848673828125, 'epoch': 0.08881941237076775})

In [16]:
import torch_xla.debug.metrics as met
print(met.short_metrics_report())
met.clear_all()

Counter: CachedCompile
  Value: 5127
Metric: CompileTime
  TotalSamples: 69
  Accumulator: 01m21s611ms438.649us
  ValueRate: 103ms558.488us / second
  Rate: 0.0877858 / second
  Percentiles: 1%=018ms806.509us; 5%=018ms962.480us; 10%=018ms977.220us; 20%=018ms102.569us; 50%=019ms684.279us; 80%=020ms692.660us; 90%=092ms417.858us; 95%=08s385ms375.178us; 99%=41s494ms927.067us
Metric: ExecuteReplicatedTime
  TotalSamples: 5196
  Accumulator: 03m23s448ms266.155us
  ValueRate: 236ms798.921us / second
  Rate: 6.70838 / second
  Percentiles: 1%=002ms666.150us; 5%=002ms038.580us; 10%=008ms481.340us; 20%=009ms722.840us; 50%=017ms692.760us; 80%=069ms464.728us; 90%=070ms776.829us; 95%=070ms038.109us; 99%=070ms435.728us
Metric: TransferToDeviceTime
  TotalSamples: 13063
  Accumulator: 01s057ms608.472us
  ValueRate: 001ms269.116us / second
  Rate: 16.4747 / second
  Percentiles: 1%=052.500us; 5%=055.780us; 10%=057.990us; 20%=060.710us; 50%=070.260us; 80%=090.770us; 90%=100.730us; 95%=111.110us; 99%=16

## Train again, this time using scan

In [12]:
from transformers import LlamaConfig, LlamaForCausalLM

reset_seed()

# Define model configuration
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,  # Model size
    num_hidden_layers=32,  # Number of transformer layers
    num_attention_heads=8,  # Number of attention heads
    intermediate_size=1024,  # Size of the hidden feedforward layer
    max_position_embeddings=128,  # Max tokens in a sequence
    use_cache=False,
    unroll_decoders=False,
)

# Instantiate the model
model = LlamaForCausalLM(config)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"].shuffle(seed=42),  # type:ignore
    eval_dataset=lm_datasets["validation"].shuffle(seed=42),  # type:ignore
    processing_class=tokenizer,
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs


NOTE: Using apply_layers to speed up compilation


Epoch,Training Loss,Validation Loss
0,6.202200,6.074419


TrainOutput(global_step=2500, training_loss=6.78487939453125, metrics={'train_runtime': 2358.5111, 'train_samples_per_second': 67.839, 'train_steps_per_second': 1.06, 'total_flos': 1.83650746368e+16, 'train_loss': 6.78487939453125, 'epoch': 0.08881941237076775})

We can see that the for loop and scan model train to the same loss over time.

In [14]:
import torch_xla.debug.metrics as met
print(met.short_metrics_report())
met.clear_all()

Counter: CachedCompile
  Value: 5127
Metric: CompileTime
  TotalSamples: 69
  Accumulator: 41s922ms465.724us
  ValueRate: 017ms411.805us / second
  Rate: 0.0293583 / second
  Percentiles: 1%=014ms787.790us; 5%=014ms218.589us; 10%=014ms466.060us; 20%=015ms696.840us; 50%=015ms103.810us; 80%=016ms766.180us; 90%=038ms692.209us; 95%=05s339ms505.876us; 99%=11s852ms781.887us
Metric: ExecuteReplicatedTime
  TotalSamples: 5196
  Accumulator: 04m12s092ms616.193us
  ValueRate: 106ms616.496us / second
  Rate: 2.41193 / second
  Percentiles: 1%=002ms282.140us; 5%=003ms828.120us; 10%=008ms281.590us; 20%=009ms508.559us; 50%=015ms485.620us; 80%=090ms391.318us; 90%=090ms472.928us; 95%=091ms562.748us; 99%=091ms792.338us
Metric: TransferToDeviceTime
  TotalSamples: 133889
  Accumulator: 15s458ms597.766us
  ValueRate: 005ms992.141us / second
  Rate: 40.6463 / second
  Percentiles: 1%=070.580us; 5%=077.270us; 10%=079.650us; 20%=083.230us; 50%=097.390us; 80%=145.530us; 90%=185.180us; 95%=242.650us; 99%=459.

## Verify the numerical correctness of `apply_layers`

Under the same weights, and the same input tokens, both the for loop based
implementation and `apply_layers` based implementation should produce the same
output tokens.

In [15]:
import torch_xla
input_ids = torch.tensor(tokenized_datasets["train"][3]["input_ids"]).unsqueeze(0).type(torch.LongTensor) # type:ignore
attention_mask = torch.tensor(tokenized_datasets["train"][3]["attention_mask"]).unsqueeze(0) # type:ignore
input_ids = input_ids.to(torch_xla.device())
attention_mask = attention_mask.to(torch_xla.device())
torch_xla.sync()

In [16]:
input_ids

tensor([[128000,   5476,     73,  56761,    912,  86262,     88,   4298,    220,
             18,    551,    366,   3200,     29,  66416,    320,  11002,    551,
          50534,     99,  75267,  16144, 115687,  33710, 123283, 104612,     18,
           1174,  13318,    662,  86262,     88,   4298,    315,    279,  71735,
            220,     18,    883,   1174,  17037,  14183,    311,    439,  86262,
             88,   4298,  66416,  14767,   4994,   6457,   1174,    374,    264,
          39747,   3560,    571,     12,     31,   5737,   2835,   1847,   8040,
            555,  80949,    323,   7972,   5168,   1854,    369,    279,  32365,
          42585,    662,  45894,    304,   6186,    220,    679,     16,    304,
           6457,   1174,    433,    374,    279,   4948,   1847,    304,    279,
          86262,     88,   4298,   4101,    662,  21445,    287,    279,   1890,
          37608,    315,  39747,    323,   1972,    571,     12,     31,    892,
          27120,    439,   1

In [17]:
model.model.unroll_decoders = False
model.model.logged_messages = set()
logits = model.forward(input_ids, attention_mask).logits  # type:ignore
logits.shape, logits

NOTE: Using apply_layers to speed up compilation


(torch.Size([1, 128, 128000]),
 tensor([[[-4.4933, -4.3954, -3.4201,  ..., -3.4511, -4.3217, -3.8000],
          [-5.2261, -4.5482, -4.1086,  ..., -4.5811, -4.8370, -4.5229],
          [-4.9636, -4.4915, -3.7757,  ..., -4.4788, -4.4865, -4.2921],
          ...,
          [-5.1460, -4.6022, -4.4547,  ..., -5.0204, -4.9061, -4.6020],
          [-4.8599, -4.3751, -4.1500,  ..., -4.9346, -4.6417, -4.5007],
          [-5.8086, -5.2585, -5.1723,  ..., -5.3529, -5.6692, -4.9910]]],
        device='xla:0', grad_fn=<UnsafeViewBackward0>))

In [18]:
def pick_token(logits):
  return torch.argmax(logits, dim=-1)

In [19]:
tokens = pick_token(logits)
tokens

tensor([[ 284,  364, 1174,  320,   13,  304, 1174, 1174,   16, 1389,  220, 3200,
           29,  720, 1174,  220,  366,  366,  883,  883,  883,  883,  883,  883,
          883,  883,  883,  220,  883,  720, 1174, 1174,  883,  279,  220,  315,
         1049, 1389,  720,  279,  220,  311,  279,  279,  304, 1174,  662,  662,
         1174,  279,  662,  323,  264,  220,  571,  315,   12,   31,  220,  323,
          662,  662,  304,  279,  662,  279,  662,   13,  662,  279,  220,  220,
          662,  720,  279,  279,  220, 1049,   15, 1174,  220, 1174,  279,  574,
          264,  220,  315,  315,  279,  220, 1174, 1174,  662,  662,  720,  574,
          279,  220,  220, 1174,  279, 1174,  279,  571,   12,   31,  220, 1174,
         1174,  264, 1176, 1174,  323,  220,  574,  304,  311,  279,  220,  220,
          662,  279,  279,  220,  366,  330,  330,  662]], device='xla:0')

In [20]:
tokenizer.decode(tokens[0].detach().cpu().numpy().tolist())

' = \', (. in,,1 – unk> \n,  < < ) ) ) ) ) ) ) ) )  ) \n,, ) the  of200 – \n the  to the the in,.., the. and a  @ of-@  and.. in the. the... the  . \n the the 2000, , the was a  of of the ,,.. \n was the  , the, the @-@ ,, a first, and  was in to the  . the the  < " ".'

In [21]:
model.model.unroll_decoders = True
model.model.logged_messages = set()
for_loop_logits = model.forward(input_ids, attention_mask).logits  # type:ignore
for_loop_logits.shape, for_loop_logits

NOTE: Using for loop to run decoder layers


(torch.Size([1, 128, 128000]),
 tensor([[[-4.4933, -4.3954, -3.4201,  ..., -3.4511, -4.3217, -3.8000],
          [-5.2261, -4.5482, -4.1086,  ..., -4.5811, -4.8370, -4.5229],
          [-4.9636, -4.4915, -3.7757,  ..., -4.4788, -4.4865, -4.2921],
          ...,
          [-5.1460, -4.6022, -4.4547,  ..., -5.0204, -4.9061, -4.6020],
          [-4.8599, -4.3751, -4.1500,  ..., -4.9346, -4.6417, -4.5007],
          [-5.8086, -5.2585, -5.1723,  ..., -5.3529, -5.6692, -4.9910]]],
        device='xla:0', grad_fn=<UnsafeViewBackward0>))

In [22]:
for_loop_tokens = pick_token(for_loop_logits)
for_loop_tokens

tensor([[ 284,  364, 1174,  320,   13,  304, 1174, 1174,   16, 1389,  220, 3200,
           29,  720, 1174,  220,  366,  366,  883,  883,  883,  883,  883,  883,
          883,  883,  883,  220,  883,  720, 1174, 1174,  883,  279,  220,  315,
         1049, 1389,  720,  279,  220,  311,  279,  279,  304, 1174,  662,  662,
         1174,  279,  662,  323,  264,  220,  571,  315,   12,   31,  220,  323,
          662,  662,  304,  279,  662,  279,  662,   13,  662,  279,  220,  220,
          662,  720,  279,  279,  220, 1049,   15, 1174,  220, 1174,  279,  574,
          264,  220,  315,  315,  279,  220, 1174, 1174,  662,  662,  720,  574,
          279,  220,  220, 1174,  279, 1174,  279,  571,   12,   31,  220, 1174,
         1174,  264, 1176, 1174,  323,  220,  574,  304,  311,  279,  220,  220,
          662,  279,  279,  220,  366,  330,  330,  662]], device='xla:0')

In [23]:
tokenizer.decode(for_loop_tokens[0].detach().cpu().numpy().tolist())

' = \', (. in,,1 – unk> \n,  < < ) ) ) ) ) ) ) ) )  ) \n,, ) the  of200 – \n the  to the the in,.., the. and a  @ of-@  and.. in the. the... the  . \n the the 2000, , the was a  of of the ,,.. \n was the  , the, the @-@ ,, a first, and  was in to the  . the the  < " ".'

In [ ]:
torch.allclose(logits, for_loop_logits, atol=1e-5, rtol=1e-5)

True

# Test the gradients of scan

After I run both scan and for loop versions of the model on the same input, their
gradients should also be similar.

In [26]:
torch_xla.sync()

input_ids.requires_grad_(False)
attention_mask.requires_grad_(False)
labels = input_ids[:, :].clone().contiguous()

# Run for loop model and collect the gradients
reset_seed()
for_loop_grads = []
model.zero_grad()
model.model.zero_grad()
torch_xla.sync()
model.model.unroll_decoders = True
model.model.logged_messages = set()
with torch.enable_grad():
  model(input_ids, attention_mask, labels=labels).loss.backward()  # type: ignore
torch_xla.sync()
for (name, param) in model.named_parameters():
  assert param.grad is not None
  for_loop_grads.append((name, param.grad.clone().detach()))

# Run scan model and collect the gradients
reset_seed()
scan_grads = []
model.zero_grad()
model.model.zero_grad()
torch_xla.sync()
model.model.unroll_decoders = False
model.model.logged_messages = set()
with torch.enable_grad():
  model(input_ids, attention_mask, labels=labels).loss.backward()  # type: ignore
torch_xla.sync()
for (name, param) in model.named_parameters():
  assert param.grad is not None
  scan_grads.append((name, param.grad.clone().detach()))

NOTE: Using for loop to run decoder layers
NOTE: Using apply_layers to speed up compilation


In [27]:
# Compare the gradients
assert len(for_loop_grads) == len(scan_grads)
assert len(for_loop_grads) > 0

In [34]:
for ((for_loop_name, for_loop_grad), (scan_name, scan_grad)) in zip(for_loop_grads, scan_grads):
  assert for_loop_name == scan_name
  assert torch.allclose(for_loop_grad, scan_grad, atol=1e-5, rtol=1e-5), f"{for_loop_name} mismatch by: {torch.max(torch.abs(for_loop_grad - scan_grad))}"
  print(f"Pass: {for_loop_name}")

Pass: model.embed_tokens.weight
Pass: model.layers.0.self_attn.q_proj.weight
Pass: model.layers.0.self_attn.k_proj.weight
Pass: model.layers.0.self_attn.v_proj.weight
Pass: model.layers.0.self_attn.o_proj.weight
Pass: model.layers.0.mlp.gate_proj.weight
Pass: model.layers.0.mlp.up_proj.weight
Pass: model.layers.0.mlp.down_proj.weight
Pass: model.layers.0.input_layernorm.weight
Pass: model.layers.0.post_attention_layernorm.weight
Pass: model.layers.1.self_attn.q_proj.weight
Pass: model.layers.1.self_attn.k_proj.weight
Pass: model.layers.1.self_attn.v_proj.weight
Pass: model.layers.1.self_attn.o_proj.weight
Pass: model.layers.1.mlp.gate_proj.weight
Pass: model.layers.1.mlp.up_proj.weight
Pass: model.layers.1.mlp.down_proj.weight
Pass: model.layers.1.input_layernorm.weight
Pass: model.layers.1.post_attention_layernorm.weight
Pass: model.layers.2.self_attn.q_proj.weight
Pass: model.layers.2.self_attn.k_proj.weight
Pass: model.layers.2.self_attn.v_proj.weight
Pass: model.layers.2.self_attn.

In [29]:
for_loop_grads[3], torch.max(for_loop_grads[3][1]), torch.min(for_loop_grads[3][1])

(('model.layers.0.self_attn.v_proj.weight',
  tensor([[-2.0387e-03,  2.9268e-03,  2.2655e-03,  ..., -8.6698e-05,
            2.9253e-03, -2.0951e-03],
          [ 1.3585e-03,  6.2552e-03, -3.5560e-03,  ...,  5.9593e-03,
            5.8606e-04,  3.1323e-04],
          [ 1.1323e-03,  5.4640e-03,  5.5192e-04,  ...,  2.8118e-03,
           -2.1846e-03,  1.2042e-03],
          ...,
          [ 1.7723e-03,  1.7468e-03, -9.1277e-04,  ...,  3.8791e-05,
           -3.4969e-04, -1.5257e-03],
          [ 2.5581e-03,  7.8343e-03, -3.5407e-03,  ...,  7.5814e-03,
            2.6671e-03, -1.4380e-03],
          [ 3.7069e-03,  5.2893e-03, -2.6598e-03,  ...,  1.6263e-03,
           -2.9556e-03,  1.9197e-03]], device='xla:0')),
 tensor(0.0229, device='xla:0'),
 tensor(-0.0295, device='xla:0'))

In [30]:
scan_grads[3], torch.max(scan_grads[3][1]), torch.min(scan_grads[3][1])

(('model.layers.0.self_attn.v_proj.weight',
  tensor([[-2.0387e-03,  2.9268e-03,  2.2655e-03,  ..., -8.6698e-05,
            2.9253e-03, -2.0951e-03],
          [ 1.3585e-03,  6.2552e-03, -3.5560e-03,  ...,  5.9593e-03,
            5.8606e-04,  3.1323e-04],
          [ 1.1323e-03,  5.4640e-03,  5.5192e-04,  ...,  2.8118e-03,
           -2.1846e-03,  1.2042e-03],
          ...,
          [ 1.7723e-03,  1.7468e-03, -9.1277e-04,  ...,  3.8791e-05,
           -3.4969e-04, -1.5257e-03],
          [ 2.5581e-03,  7.8343e-03, -3.5407e-03,  ...,  7.5814e-03,
            2.6671e-03, -1.4380e-03],
          [ 3.7069e-03,  5.2893e-03, -2.6598e-03,  ...,  1.6263e-03,
           -2.9556e-03,  1.9197e-03]], device='xla:0')),
 tensor(0.0229, device='xla:0'),
 tensor(-0.0295, device='xla:0'))